### Utility functions

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from headers import *
from spacy.vocab import Vocab
from spacy.language import Language
import spacy

In [ ]:
def readdict(filename):
    print "Extracting the dictionary from %s"%(filename)
    with open(filename,'rb') as handle:
        dictionary=pickle.load(handle)

    wcount=len(dictionary)
    return dictionary,wcount

In [ ]:
dictionary,wcount=readdict(WNDICT)

In [ ]:
def remove_swords(ltokens):
    res=[w for w in ltokens if w not in SWORDS]
    return res

In [ ]:
def tok_sentence(w,m):
    sentence=w+' '+m
    tokenized_sentence=sentence.split()
    return remove_swords(tokenized_sentence)
    

In [ ]:
avg=0.0
local_vocab=set()
for w,m in dictionary:
    #print tok_sentence(w,m)
    #local_vocab.update(tok_sentence(w,m))
    avg+=len(m.split(' '))
    local_vocab.add(w)
print avg/len(dictionary)
print len(local_vocab)
print len(global_vocab)
if 'blarina'in set(local_vocab):
    print True

In [ ]:
dict2=[x for x in dictionary if nlp(unicode(x[0],'utf-8')).has_vector and nlp(unicode(x[0],'utf-8')).vector.all()!=0]
len(dict2)

max_sentence=0
ms=[]
for i in range(len(dict2)):
    tok=dict2[i][1].split()
    if len(tok)>DEF_LEN:
        new_def=' '.join(tok[:DEF_LEN])
        dict2[i]=(dict2[i][0],new_def)
        #print "Removed",len(dict2[i][1].split())
    elif len(tok)<DEF_LEN:
        diff=DEF_LEN-len(tok)
        for x in range(diff):
            tok.append('<PAD>')
        new_def=' '.join(tok)
        dict2[i]=(dict2[i][0],new_def)
        #print "Added",len(dict2[i][1].split())

In [ ]:
Y_set=[]
X_set=np.zeros((len(dict2),DEF_LEN,WV_SIZE))
Y_GT=[]

for i in range(len(dict2)):
    w=dict2[i][0]
    m=dict2[i][1]
    #check for the word
    docw=nlp(unicode(w,'utf-8'))
    Y_set.append(docw[0].vector)
    Y_GT.append(w)
    #check for all the words in the meaning
    tok=m.split()
    for t in range(len(tok)):
        docm=nlp(unicode(tok[t],'utf-8'))
        if docm[0].has_vector and docm[0].vector.all()!=0:
            X_set[i,t,:]=docm[0].vector
        else:
            X_set[i,t,:]=np.zeros((WV_SIZE,))
Y_set=np.array(Y_set)
print Y_set.shape
print len(Y_GT)
print X_set.shape

In [ ]:
X_train,X_val,X_test,Y_train,Y_val,Y_test=X_set[:-2000],X_set[-2000:-1000],X_set[-1000:],Y_set[:-2000],Y_set[-2000:-1000],Y_set[-1000:]
print X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape

In [ ]:
def readwe(filename):
    with open(filename,'rb') as handle:
        nlp=pickle.load(handle)
    return nlp

In [ ]:
def return_whole_file(filename):
    dict,wcount=readdict(filename)
    whole_thing=''
    for word,meaning in dict:
        #removed word from the return string
        whole_thing+=' '+meaning
    return whole_thing

In [ ]:
def writeemb(word_embeddings,filename):
    with open(filename,'wb') as handle:
        pickle.dump(word_embeddings,handle,protocol=pickle.HIGHEST_PROTOCOL)

    print "Done writing the word_embeddings to %s"%(filename)

In [ ]:
def reademb(filename):
    print "Extracting the word_embeddings from %s"%(filename)
    with open(filename,'rb') as handle:
        word_embeddings=pickle.load(handle)

    return word_embeddings

In [ ]:
def tokenize(dict):
    #dict is a list of (word, definitions)
    #Every word-definition pair is converted to a sentence by concatenating the word+definition
    #This sentence is tokenized to get a list of tokens for every word-definition pair
    #All such lists are appended to tok
    tok=[]
    for i,(w,d) in enumerate(dict):
        #removed word from the token list
        sentence=w+' '+d
        tokenized_sentence=sentence.split()
        tok.append(remove_swords(tokenized_sentence))
    return tok,len(tok)

In [ ]:
def train_test_split(dict,ratio):
    N=len(dict)
    train_size=ratio
    random.shuffle(dict)
    train_set=dict[:-train_size]
    test_set=list(set(dict)-set(train_set))
    return train_set,test_set

In [ ]:
def make_cumm_embedding(input_sentence,method):
    N=WV_SIZE
    doc=nlp(input_sentence)
    if method=='additive':
        res=np.zeros((N,))
        for i in range(len(doc)):
            if not doc[i].has_vector:
                continue
            if doc[i].vector.all()!=0:
                res+=doc[i].vector
    elif method=='multiplicative':
        res=np.ones((N,))
        for i in range(len(doc)):
            if not doc[i].has_vector:
                continue
            if doc[i].vector.all()!=0:
                res*=doc[i].vector
    return res

In [ ]:
def cosine_sim(word_embedding1,word_embedding2):
    d1=np.sqrt(np.sum(word_embedding1**2))
    d2=np.sqrt(np.sum(word_embedding2**2))
    n=word_embedding1.dot(word_embedding2)
    return n/(d1*d2)
    

In [ ]:
def sq_distance(word_embedding1,word_embedding2):
    return tf.sqrt(tf.reduce_sum((word_embedding1-word_embedding2)**2,0))

In [ ]:
def get_most_similar(input_embedding):
    cs_list=[]
    for v in set(local_vocab):
        doc=nlp(unicode(v,'utf-8'))
        if doc[0].has_vector:
            if doc[0].vector.all()!=0.0:
                cs=cosine_sim(input_embedding,doc[0].vector)
                if not math.isnan(cs):
                    cs_list.append((doc[0].text,cs))
            else:
                s=np.array([1e-5 for x in range(WV_SIZE)])
                cs=cosine_sim(input_embedding,s)
                if not math.isnan(cs):
                    cs_list.append((doc[0].text,cs))
                
                
                    
    sorted_cs_list=sorted(cs_list,key=lambda tup:tup[1],reverse=True)
    return sorted_cs_list

In [ ]:
def get_most_similar_for_lstm(input_embedding):
    cs_list=[]
    for v in global_vocab:
        doc=nlp(v)
        if doc[0].has_vector:
            if doc[0].vector.all()!=0.0:
                cs=cosine_sim(input_embedding,doc[0].vector)
                if not math.isnan(cs):
                    cs_list.append((doc[0].text,cs))
            else:
                s=np.array([1e-5 for x in range(WV_SIZE)])
                cs=cosine_sim(input_embedding,s)
                if not math.isnan(cs):
                    cs_list.append((doc[0].text,cs))
                
                
                    
    sorted_cs_list=sorted(cs_list,key=lambda tup:tup[1],reverse=True)
    return sorted_cs_list

In [ ]:
def get_list_from_tuple(ltuple):
    res=[]
    scores=[]
    for k,v in ltuple:
        res.append(k)
        scores.append(v)
    return res,scores

In [ ]:
def get_rank_bins(ranks):
    #500,1000,5000,10000
    
    N=len(BIN_VALS)
    bins={x:0 for x in BIN_VALS}
    for i in ranks:
        if i<=500:
            bins[500]+=1
        elif i<=1000 and i>500:
            bins[1000]+=1
        elif i<=5000 and i>1000:
            bins[5000]+=1
        elif i<=10000 and i>5000:
            bins[10000]+=1
    return bins

In [ ]:
#Metric 1: % of words present in the top recall_count results
#Metric 2: mean rank of the target words

def eval_method_with_spacy(test_set,recall_count,method='additive'):
    recall_size=0
    acc=0
    i=0
    ranks=[]
    for t in test_set:
        i+=1
        target_word=nlp(unicode(t[0]),'utf-8')
        input_sentence=unicode(t[1],'utf-8')
        input_embedding=make_cumm_embedding(input_sentence,method)
        if input_embedding.all()==0:
            continue
        r=get_list_from_tuple(get_most_similar(input_embedding))
        results=r[0]
        scores=r[1]
        if target_word[0].text in results:
            print target_word[0].text
            print "There"
        print input_sentence,' ',target_word[0].text,'/',results[0],' ',scores[0]
        if target_word[0].text in results:
            target_rank=results.index(target_word[0].text)
            ranks.append(target_rank)
            print "%d/%d done for word %s, rank: %d"%(i,len(test_set),str(target_word[0].text),target_rank)
    print "Mean rank: %f"%(float(sum(ranks))/float(len(ranks)))
    print get_rank_bins(ranks)
        